In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import joblib
import time
import itertools
import requests
from bs4 import BeautifulSoup
import time
import os
import pandas as pd
import codecs
from urllib.parse import urljoin

In [2]:
def joblib_url(i,z):
    url = 0
    new_list = list()
    urls = 'https://townwork.net/joSrchRsltList/?jc={0:03d}&page={1}'.format(i,z)
    res = requests.get(urls)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    divs = soup.find_all('div',class_='job-lst-box-wrap')
    for div in divs:
        link = div.find('a')
        href = link.attrs['href'] 
        new_list.append(href)
    return new_list
            

In [ ]:
all_list = list()
a=range(1,100)
b=range(1,4000)
for i in a:
    try:
        resultList = joblib.Parallel(n_jobs=-1, verbose=3)( [joblib.delayed(joblib_url)(i,z) for z  in b])
    except:
        pass
    all_list.extend(resultList)    

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:   27.4s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   54.5s
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed:  1.6min


In [ ]:
new_all_list = list(itertools.chain.from_iterable(all_list))

In [ ]:
kekka = pd.DataFrame(new_all_list,columns=["url"])
kekka

In [ ]:
url_key = kekka["url"]

In [ ]:
url_key

In [ ]:
def joblib_get_data(z):
    url = houjinnmei = shokushu = gyoukai = juusho = dennwabanngou = Web_url = ""
    new_list = list()
    url = "https://townwork.net/"+url_key[z]
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    k=[]
    for i in range(500):
        try:
            juusho = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(1) > dd > p ".format(i)).get_text()
            k.append(i)
       
        except:
            pass
    kk=max(k,default=10)
    try:
        houjinnmei = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(1) > dd > p".format(kk)).get_text()
    except:
        houjinnmei = ""
    try:
        shokushu = soup.select_one("#js-detailArea > div.job-detail-box-inner > div.job-detail-box-container > div.job-detail-tbl-wrap.job-detail-tbl-main-wrap > dl:nth-child(1) > dd > p > span").get_text()
    except:
        shokushu = ""
    try:
        juusho = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(3) > dd > p".format(kk)).get_text()
    except:
        juusho = ""
    try:
        dennwabanngou = soup.select_one("#js-detailArea > form > div > ul > li.detail-tel-box.jsc-phone-popup > p.detail-tel-ttl > span").get_text().split()
    except:
        dennwabanngou = ""
    try:
        gyoukai = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(2) > dd > p".format(kk)).get_text()
    except:
        gyoukai = ""
    try:
        Web_url = soup.select_one("#js-detailArea > div:nth-child({}) > div > dl:nth-child(4) > dd > a".format(kk)).get_text()
    except:
        Web_url = ""
    new_list.append(houjinnmei)
    new_list.append(shokushu)
    new_list.append(gyoukai)
    new_list.append(juusho)
    new_list.append(dennwabanngou)
    new_list.append(Web_url)
    new_list.append(url)
    return new_list

In [ ]:
all_list = ()
resultList = joblib.Parallel(n_jobs=-1, verbose=3)([joblib.delayed(joblib_get_data)(z) for z in range(len(url_key))])
all_list.extend(resultList)

In [ ]:
df_kekka = pd.DataFrame(all_list,columns=["法人名","職種","業界","住所","電話番号","webURL","ページurl"])
df_kekka